In [1]:
# Prwto thema

# Import necessary libraries for web scraping, browser automation, and data handling
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

# Ask the user for the URL and tag name, build filenames for output CSVs
url = input('Give me eplease the URL')
tname = input('print the tag name')
a = 'Prwto_Thema'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

# Launch Firefox browser using WebDriver manager
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Lists to hold dates and tags from articles
date_list = []
tag_list = []

# The cutoff date - once reached, scraping will stop
stop_date = 20250431
scroll_pause_time = 2.6

# Open the initial URL in the browser
driver.get(url)

# Give the page time to load and maximize window
time.sleep(2)
driver.maximize_window()
time.sleep(1)

# Accept cookies or dismiss popup by clicking "disagree" button
disagree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

# Flags used to control scraping flow
flag = True
inside_flag = True

# Function to handle scrolling and checking dates while scraping
def scroll(driver, timeout):
    global flag
    global stop_date
    global inside_flag
    global tag_list
    global date_list
    scroll_pause_time = timeout

    # Number of scrolls before checking the date again
    nr_of_scrollings = 2
    changable_nr_of_scrollings = nr_of_scrollings

    # Check if the date of the latest articles is older than the stop date
    def date_check_fun():
        global inside_flag

        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'lxml')
        articles_place = soup.find('div' , class_ = "listLoop")
        dates = articles_place.find_all('div' , class_ = "wrp")

        # Extract date from last article block
        for i in dates:
            date = i.find('time')
            date = date.get_text()
            date = date.split(",")
            date = date[0].split(".")
            date.reverse()
            date = "".join(date)
            date = int(date)

        # If date is before the stop date, stop scraping
        if date < stop_date:
            print('the date is smaller. Programm Must STOP')
            inside_flag = False
            return inside_flag

    # Initial scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while inside_flag:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        # Check if more content is loaded
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print('the page reaches the end for the first time. It cannot scroll anymore')
            break
        last_height = new_height
        time.sleep(scroll_pause_time)

        # After every few scrolls, check article dates
        changable_nr_of_scrollings -= 1
        if changable_nr_of_scrollings == 0:
            date_check_fun()
            changable_nr_of_scrollings = nr_of_scrollings

    # Function to extract tags from individual article pages
    def tag_fun(inside):
        global tag_list
        inside_tag_list = []

        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')
        tags = soup.find('div' , class_ = "tagsCnt")
        tags = tags.find_all('a')

        # Clean and collect each tag
        for a in tags:
            tag = a.get_text().replace('\r', "").replace('\n', "").strip()
            inside_tag_list.append(tag)

        tag_list.append(inside_tag_list)

    # After scrolling, extract article URLs and dates
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'lxml')

    articles_place = soup.find('div' , class_ = "listLoop")
    articles = soup.find_all('article')

    for i in articles:
        # Extract and convert date
        date = i.find('div' , class_ = "wrp").find('time').get_text()
        date = date.split(",")[0].split(".")
        date.reverse()
        date = int("".join(date))

        # If date is older than stop_date, stop
        if date < stop_date:
            print('the date is smaller. I need to terminate the whole programm')
            break
        else:
            # Get full article URL
            article_url = i.find('div', class_='heading').find('a').get('href')
            https = "https://www.protothema.gr"
            if article_url.find(https) < 0:
                article_url = https + article_url
                print('apo katw article url ' , article_url)

            date_list.append(date)
            tag_fun(article_url)

# Start scrolling and scraping process
scroll(driver , scroll_pause_time)

# Print tag and date info
print(tag_list)
print(len(tag_list))
print(len(date_list))
print('the last date is ' , date_list[-1])

# ----------- PROCESS AND SAVE TAG FREQUENCIES -----------
import pandas as pd

# Flatten tag list into a single list
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        new_tag_list.append(tag_list[i][j])

# Count frequency of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)
print(res)

# Convert dictionary to list of key-value pairs
resultList = list(map(list, res.items()))
print(resultList)

# Save tag counts to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# ----------- PREPARE DATE-TAG PAIRS FOR EXPORT -----------

# Create a dot-formatted version of the dates (dd.mm.yyyy)
ndlist = []
for i in date_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(date_list)
print(ndlist)

# Create a list of [tag, date] for all tags
n = 0
date_con_list = []
for i in tag_list:
    n += 1
    for tag in i:
        inside = [tag, date_list[n-1]]
        date_con_list.append(inside)

print(date_con_list)

# Save tag-date pairs to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final debug output
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


the date is smaller. Programm Must STOP
the date is smaller. I need to terminate the whole programm
[['ΠΟΛΙΤΙΚΗ', 'Δημοσκόπηση Interview', 'Νέα Δημοκρατία', 'ΠΑΣΟΚ', 'Κυριάκος Μητσοτάκης', 'Αλέξης Τσίπρας', 'Αντώνης Σαμαράς', 'Δημοσκόπηση'], ['ΠΟΛΙΤΙΚΗ', 'Δημοσκόπηση', 'Δημοσκόπηση Opinion Poll', 'Νέα Δημοκρατία', 'Αλέξης Τσίπρας', 'Πλεύση Ελευθερίας', 'ΠΑΣΟΚ'], ['ΕΛΛΑΔΑ', 'Κύπρος', 'Βλαντιμίρ Πούτιν', 'Σι Τζινπίνγκ', 'Δημοσκόπηση'], ['ΕΛΛΑΔΑ', 'Βία ανηλίκων', 'Μέτρα', 'Πανεπιστήμια', 'Δημοσκόπηση', 'Βία στα Πανεπιστήμια'], ['ΠΟΛΙΤΙΚΗ', 'Δημοσκόπηση', 'Γκάλοπ', 'Νέα Δημοκρατία', 'ΠΑΣΟΚ'], ['ΠΟΛΙΤΙΚΗ', 'Δημοσκόπηση', 'Νέα Δημοκρατία', 'Πλεύση Ελευθερίας', 'ΠΑΣΟΚ'], ['ΠΟΛΙΤΙΚΗ', 'Δημοσκόπηση Interview', 'Νέα Δημοκρατία', 'Κυριάκος Μητσοτάκης', 'ΠΑΣΟΚ', 'Πλεύση Ελευθερίας', 'Δημοσκόπηση'], ['ΚΟΣΜΟΣ', 'Πόλεμος στην Ουκρανία', 'Δημοσκόπηση', 'Ρωσία', 'Βιτάλι Κλίτσκο', 'Ντόναλντ Τραμπ'], ['ΚΟΣΜΟΣ', 'Γαλλία', 'Δημοσκόπηση', 'Εκλογές', 'Γαλλικές εκλογές', 'Προεδρικές εκλογές']]
9
9
the last da